In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
from sklearn.model_selection import KFold
import scipy as sc

import training_utils


In [2]:
img_size = (240, 320)
img_channels = 3
batch_size = 128
epochs = 50
display_epochs = (0, 100)
inputs = keras.Input(shape= img_size + (img_channels,))


x_col_name = 0
y_col_name = 1

In [3]:
model_name = 'VGG16_(256).tf'
global_history= {}


In [4]:
dataset_path = '/Users/olove/Library/CloudStorage/OneDrive-Personal/AI datasets/jhu_crowd_v2.0'


In [5]:
callbacks_list = [
    #    keras.callbacks.EarlyStopping(
    #        monitor="val_loss", patience=4
    #    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name,
        monitor="val_loss",
        save_best_only=True
    ),
    #   keras.callbacks.TensorBoard()
]

In [6]:
train_df = pd.read_csv(dataset_path + '/train/image_labels.txt', sep=',', header=None, dtype={0: str, 1: np.float32})
val_df = pd.read_csv(dataset_path + '/val/image_labels.txt', sep=',', header=None, dtype={0: str, 1: np.float32})
test_df = pd.read_csv(dataset_path + '/test/image_labels.txt', sep=',', header=None, dtype={0: str, 1: np.float32})

In [7]:
display(train_df.head())

,0,1,2,3,4
0,0001,161.0,water park,0,0
1,0008,855.0,marathon,0,0
2,0010,455.0,protest,0,0
3,0012,7139.0,stadium,0,0
4,0014,111.0,protest,0,0


In [8]:
train_df.columns

Index([0, 1, 2, 3, 4], dtype='int64')

In [9]:
train_df[0] = train_df[0].apply(lambda x: str(x) + '.jpg')
val_df[0] = val_df[0].apply(lambda x: str(x) + '.jpg')
test_df[0] = test_df[0].apply(lambda x: str(x) + '.jpg')

In [10]:
display(train_df.head())
display(val_df.head())
display(test_df.head())

,0,1,2,3,4
0,0001.jpg,161.0,water park,0,0
1,0008.jpg,855.0,marathon,0,0
2,0010.jpg,455.0,protest,0,0
3,0012.jpg,7139.0,stadium,0,0
4,0014.jpg,111.0,protest,0,0


,0,1,2,3,4
0,0003.jpg,804.0,street,0,0
1,0007.jpg,41.0,street,0,0
2,0009.jpg,26.0,street,1,0
3,0011.jpg,31.0,stadium,0,0
4,0013.jpg,5000.0,stadium,0,0


,0,1,2,3,4
0,0002.jpg,43.0,street,1,0
1,0004.jpg,59.0,street,0,0
2,0005.jpg,463.0,stadium,0,0
3,0006.jpg,299.0,conference,0,0
4,0016.jpg,140.0,rally,0,0


In [11]:
def load_single_generator(df, img_path='/frames/frames/', debug=False):
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    datagen = ImageDataGenerator(
        rescale=1. / 255,
    )
    if debug:
        display(df)
        print(x_col_name)
        print(y_col_name)

    print(''.join([dataset_path, img_path]))
    generator = datagen.flow_from_dataframe(
        df,
        ''.join([dataset_path, img_path]),
        x_col=x_col_name,
        y_col=y_col_name,
        class_mode='raw',
        target_size=img_size,
        batch_size=batch_size,
    )

    return generator


In [12]:
train_generator = load_single_generator(train_df, img_path='/train/images/')
val_generator = load_single_generator(val_df, img_path='/val/images/')
test_generator = load_single_generator(test_df, img_path='/test/images/')

/Users/olove/Library/CloudStorage/OneDrive-Personal/AI datasets/jhu_crowd_v2.0/train/images/
Found 2272 validated image filenames.
/Users/olove/Library/CloudStorage/OneDrive-Personal/AI datasets/jhu_crowd_v2.0/val/images/
Found 500 validated image filenames.
/Users/olove/Library/CloudStorage/OneDrive-Personal/AI datasets/jhu_crowd_v2.0/test/images/
Found 1600 validated image filenames.


In [13]:
baseline_mae = np.mean(np.abs(test_df[1] - np.mean(test_df[1])))
print(f'Baseline MAE: {baseline_mae}')
baseline_mae = np.mean(np.abs(val_df[1] - np.mean(val_df[1])))
print(f'Baseline MAE: {baseline_mae}')
baseline_mae = np.mean(np.abs(train_df[1] - np.mean(train_df[1])))
print(f'Baseline MAE: {baseline_mae}')



Baseline MAE: 345.35308837890625
Baseline MAE: 332.0926818847656
Baseline MAE: 425.99920654296875


# Models

## Simple Covnet model

In [14]:
def basic_covnet_block():
    x = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation="relu")(inputs)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=2, activation="relu")(x)
    x = layers.Conv2D(filters=128, kernel_size=3, strides=2, activation="relu")(x)
    return x

In [15]:
covnet_block = basic_covnet_block()
x = layers.Flatten()(covnet_block)
x = layers.Dense(1)(x)
model = keras.Model(inputs=inputs, outputs=x)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 320, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 119, 159, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 59, 79, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 39, 128)       73856     
                                                                 
 flatten (Flatten)           (None, 144768)            0         
                                                                 
 dense (Dense)               (None, 1)                 144769    
                                                                 
Total params: 238017 (929.75 KB)
Trainable params: 238017 (92

2024-01-04 22:19:33.585463: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-01-04 22:19:33.585484: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-04 22:19:33.585492: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-04 22:19:33.585673: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-04 22:19:33.585853: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [16]:
Wsave = model.get_weights()

In [17]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks = callbacks_list,
                    validation_data=val_generator,
                    )

Epoch 1/50


2024-01-04 22:19:36.835138: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - ETA: 0s - loss: 1328470.7500 - mae: 381.1823INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


18/18 [==============================] - 58s 3s/step - loss: 1328470.7500 - mae: 381.1823 - val_loss: 466580.0938 - val_mae: 375.5205
Epoch 2/50
18/18 [==============================] - ETA: 0s - loss: 1288038.2500 - mae: 434.9092INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


...

INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


18/18 [==============================] - 53s 3s/step - loss: 1032199.5625 - mae: 380.7887 - val_loss: 311082.6250 - val_mae: 290.9614
Epoch 13/50
18/18 [==============================] - 53s 3s/step - loss: 1025891.2500 - mae: 364.7167 - val_loss: 333614.9062 - val_mae: 248.9068
Epoch 14/50
18/18 [==============================] - 53s 3s/step - loss: 1021518.7500 - mae: 383.9309 - val_loss: 404778.3438 - val_mae: 251.0758
Epoch 15/50
18/18 [==============================] - 53s 3s/step - loss: 1115843.5000 - mae: 375.8294 - val_loss: 355139.2812 - val_mae: 245.6862
Epoch 16/50
18/18 [==============================] - 52s 3s/step - loss: 1028233.1250 - mae: 371.8132 - val_loss: 317993.4062 - val_mae: 302.1158
Epoch 17/50
18/18 [==============================] - 53s 3s/step - loss: 972841.8125 - mae: 353.7444 - val_loss: 317051.5312 - val_mae: 305.3275
Epoch 18/50
18/18 [==============================] - ETA: 0s - loss: 976669.5000 - mae: 374.6840 INFO:tensorflow:Assets written to: VGG16

INFO:tensorflow:Assets written to: VGG16_(256).tf/assets


18/18 [==============================] - 53s 3s/step - loss: 976669.5000 - mae: 374.6840 - val_loss: 307574.3750 - val_mae: 278.2740
Epoch 19/50
18/18 [==============================] - 53s 3s/step - loss: 965179.2500 - mae: 364.7060 - val_loss: 402632.5938 - val_mae: 387.1693
Epoch 20/50
18/18 [==============================] - 53s 3s/step - loss: 929493.9375 - mae: 347.8424 - val_loss: 330365.5625 - val_mae: 316.0418
Epoch 21/50
18/18 [==============================] - 53s 3s/step - loss: 950076.5000 - mae: 374.0968 - val_loss: 341404.4688 - val_mae: 245.5388
Epoch 22/50
18/18 [==============================] - 53s 3s/step - loss: 982655.8750 - mae: 368.3983 - val_loss: 444640.6875 - val_mae: 252.1456
Epoch 23/50
18/18 [==============================] - 52s 3s/step - loss: 1095296.6250 - mae: 359.4963 - val_loss: 395990.4062 - val_mae: 395.5632
Epoch 24/50
18/18 [==============================] - 53s 3s/step - loss: 946394.1250 - mae: 368.9588 - val_loss: 454638.4375 - val_mae: 426.3

KeyboardInterrupt: 

In [ ]:
global_history['simple_covnet_(32_64_128)_()'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

## Depthwise model

In [ ]:
def custom_depthwise_conv_block(input, sizes, repeat_per_size, stride=1, pooling=False, dropout=False, residual=False):
    x = input

    previous_block_activation = x

    for size in sizes:
        for i in range(repeat_per_size - 1):
            x = layers.BatchNormalization()(x)
            x = layers.Activation("relu")(x)
            x = layers.SeparableConv2D(size, 3, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        if dropout:
            x = layers.Dropout(0.5)(x)
        x = layers.SeparableConv2D(size, 3, strides=1 if pooling else stride, padding='same', use_bias=False)(x)
        if pooling:
            x = layers.MaxPooling2D(3, strides=stride, padding='same')(x)
        if residual:
            r = layers.SeparableConv2D(size, 1, strides=stride, padding="same")(
                previous_block_activation
            )
            x = layers.add([x, r])
            previous_block_activation = x

    return x

In [ ]:
def basic_dense_block(covnet_param, sizes, global_pooling=False):
    if global_pooling:
        x = layers.GlobalAveragePooling2D()(covnet_param)
    else:
        x = layers.Flatten()(covnet_param)
    x = layers.Dropout(0.5)(x)
    for size in sizes:
        x = layers.Dense(size, activation="relu")(x)
        x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1)(x)
    return outputs

### Basic depthwise model

#### Single Output dense layer

In [ ]:
covnet = custom_depthwise_conv_block(inputs, [32, 64, 128], 2, stride=2)
outputs = basic_dense_block(covnet, [])
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks = callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['depthwise_covnet_(32_64_128)_()'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

#### Multiple dense layers

In [ ]:
covnet = custom_depthwise_conv_block(inputs, [32, 64, 128], 2, stride=2)
outputs = basic_dense_block(covnet, [64])
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks = callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['depthwise_covnet_(32_64_128)_(64)'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)


### Depthwise model with Max pooling

#### Single Output dense layer

In [ ]:
covnet = custom_depthwise_conv_block(inputs, [32, 64, 128], 2, pooling=True)
outputs = basic_dense_block(covnet, [])
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks = callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['depthwise_covnet_pooling_(32_64_128)_()'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

#### Multiple dense layers

In [ ]:
covnet = custom_depthwise_conv_block(inputs, [32, 64, 128], 2, pooling=True)
outputs = basic_dense_block(covnet, [64])
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks = callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['depthwise_covnet_pooling_(32_64_128)_(64)'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

#### Multiple dense layers with dropout

In [ ]:
covnet = custom_depthwise_conv_block(inputs, [32, 64, 128], 2, pooling=True, dropout=True)
outputs = basic_dense_block(covnet, [64])
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks = callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['depthwise_covnet_pooling_(32_64_128)_(64)_dropout'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

### Depthwise model with residual connections

In [ ]:
covnet = custom_depthwise_conv_block(inputs, [32, 64, 128], 2, stride=2, residual=True)
outputs = basic_dense_block(covnet, [64])
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks = callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['depthwise_covnet_residual_(32_64_128)_(64)'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

In [ ]:
covnet = custom_depthwise_conv_block(inputs, [32, 64, 128], 2, stride=2, residual=True, dropout=True)
outputs = basic_dense_block(covnet, [])
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks = callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['depthwise_covnet_residual_(32_64_128)_()_dropout'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

### Depthwise model with residual connections and global pooling

In [ ]:
covnet = custom_depthwise_conv_block(inputs, [32, 64, 128], 2, stride=2, pooling=True, residual=True)
outputs = basic_dense_block(covnet, [64], global_pooling=True)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks = callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['depthwise_covnet_residual_pooling_(32_64_128)_(64)'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

In [ ]:
covnet = custom_depthwise_conv_block(inputs, [32, 64, 128], 2, stride=2, pooling=True, residual=True, dropout=True)
outputs = basic_dense_block(covnet, [64], global_pooling=True)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['depthwise_covnet_residual_pooling_(32_64_128)_(64)_dropout'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

## Xception


In [ ]:
def Xception_model():
    covnet = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(inputs)
    covnet = custom_depthwise_conv_block(covnet, [32, 64], 2, stride=2, pooling=True, residual=True)
    covnet = custom_depthwise_conv_block(covnet, [128], 2, residual=True)
    outputs = basic_dense_block(covnet, [], global_pooling=True)
    return keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model = Xception_model()
model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
model.summary()

In [ ]:
Wsave = model.get_weights()

In [ ]:
model.set_weights(Wsave)
history = model.fit(train_generator,
                    epochs=epochs,
                    callbacks=callbacks_list,
                    validation_data=val_generator,
                    )

In [ ]:
global_history['Xception'] = history

In [ ]:
training_utils.history_metric_evaluation(history, display_epochs)

## VGG16

In [ ]:
def base_vgg_16_layers(input):
    input = keras.applications.vgg16.preprocess_input(input)
    covnet = keras.applications.vgg16.VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=img_size + (img_channels,))(input)
    covnet.trainable = False
    return keras.Model(inputs=input, outputs=covnet)


def output_vgg_16_layers(covnet, node_size):
    output = basic_dense_block(covnet, node_size)
    return keras.Model(inputs=covnet, outputs=output)


def vgg_16_model():
    input = keras.applications.vgg16.preprocess_input(inputs)
    covnet = base_vgg_16_layers()(input)
    return output_vgg_16_layers(covnet, input)


def activate_fine_tuning_on_vgg_16(ft_model):
    # Flag to indicate whether the layers should be trainable
    set_trainable = False

    # Assuming 'vgg16' is the name of the nested VGG16 model
    vgg16 = ft_model.get_layer('vgg16')
    vgg16.trainable = True

    for layer in vgg16.layers:
        # Start fine-tuning from 'block5_conv1'
        if layer.name == 'block5_conv1':
            set_trainable = True

        # Set the trainable flag for the layers
        if set_trainable:
            print(f'Unfreezing layer {layer.name}')
            layer.trainable = True
        else:
            print(f'Freezing layer {layer.name}')
            layer.trainable = False


In [ ]:
conv_base = base_vgg_16_layers(keras.layers.Input(shape=img_size + (img_channels,)))
conv_base.summary()

In [ ]:
feature_train = conv_base.predict(train_generator, 2000, verbose=1)
feature_val = conv_base.predict(val_generator, 2000, verbose=1)


In [ ]:
print(feature_train.shape)
display(feature_train[0][0])
display(train_df.head())

In [ ]:
i_model = output_vgg_16_layers(keras.Input(shape=feature_train.shape[1:]), [])

i_model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
dense_history = i_model.fit(feature_train, train_df[1].values,
                            epochs=epochs,
                            callbacks=callbacks_list,
                            validation_data=(feature_val, val_df[1].values),
                            )

In [ ]:
global_history['vgg16_()_dense'] = dense_history

In [ ]:

merged_model = keras.Model(inputs=conv_base.input, outputs=i_model(conv_base.output))
activate_fine_tuning_on_vgg_16(merged_model)
merged_model.compile(loss="mse", optimizer="adam", metrics=["mae"])
merged_model.summary()

In [ ]:
conv_history = merged_model.fit(train_generator,
                                epochs=epochs,
                                callbacks=callbacks_list,
                                validation_data=val_generator,
                                )

In [ ]:
global_history['vgg16_()_conv'] = conv_history

In [ ]:
print(f'Min mae: {np.min(conv_history.history["mae"])}')
print(f'Min val_mae: {np.min(conv_history.history["val_mae"])}')
print(f'Min loss: {np.min(conv_history.history["loss"])}')
print(f'Min val_loss: {np.min(conv_history.history["val_loss"])}')

In [ ]:
training_utils.history_metric_evaluation(conv_history, display_epochs)

In [ ]:
i_model = output_vgg_16_layers(keras.Input(shape=feature_train.shape[1:]), [64])

i_model.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [ ]:
dense_history = i_model.fit(feature_train, train_df[1].values,
                            epochs=epochs,
                            callbacks=callbacks_list,
                            validation_data=(feature_val, val_df[1].values),
                            )

In [ ]:
global_history['vgg16_(64)_dense'] = dense_history

In [ ]:

merged_model = keras.Model(inputs=conv_base.input, outputs=i_model(conv_base.output))
activate_fine_tuning_on_vgg_16(merged_model)
merged_model.compile(loss="mse", optimizer="adam", metrics=["mae"])
merged_model.summary()

In [ ]:
conv_history = merged_model.fit(train_generator,
                                epochs=epochs,
                                callbacks=callbacks_list,
                                validation_data=val_df,
                                )

In [ ]:
global_history['vgg16_(64)_conv'] = conv_history

In [ ]:
training_utils.history_metric_evaluation(conv_history, display_epochs)